# Preprocessing participatory budgeting data for Cambridge, Miami and New York

# Data preparation
Before we start with the recommendation algorithms, we need to prepare the data. Firstly, we create a matrix with the users and the proposals they have commented on. This matrix will reflex how many times a user has commented on a proposal.
This matrix will be defined as follows: $M_{u,p}$ is the number of comments that user $u$ has made on proposal $p$. This matrix will be sparse, as most users have not commented on most proposals, and will be saved in a sparse matrix format to save memory. As we have a lot of different databases, we need to define the structure of the directories where the data is stored. We will define the following structure:
- `data\`: directory where the data is saved
    - `data\rm\`: directory where we save the recommendation matrices.
        - `data\rm\city\year\`: directory where we save the recommendation matrices for a city and a year.

The filenames will be defined as follows:
- `data\rm\city\year\num_comm_matrix.npz`: sparse matrix with the number of comments that each user has made on each proposal.
- `data\rm\city\year\num_comm_train.npz`: sparse matrix with the number of comments that each user has made on each proposal in the training set.
- `data\rm\city\year\num_comm_test.npz`: sparse matrix with the number of comments that each user has made on each proposal in the test set.
- `data\rm\city\year\user_mapping.csv`: mapping between each userId and its index in the matrix.
- `data\rm\city\year\item_mapping.csv`: mapping between each itemId and its index in the matrix.

**Remember**: in our database, itemId refers to the proposals and userId refers to the users.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import warnings
warnings.simplefilter('ignore')
from scipy.sparse import load_npz, csr_matrix, save_npz
import scipy.sparse as sps
# DB connection and path modification
import sys
import os
import pymysql
import numpy as np
import matplotlib.pyplot as plt


#implicit
from implicit.evaluation import train_test_split
from implicit.cpu.lmf import LogisticMatrixFactorization
from implicit.cpu.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import CosineRecommender

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    python_path = os.path.join(module_path, 'python')
    sys.path.append(python_path)

# Recommenders
from implicit_extend.popularity import PopularityRecommender, PopularityNumCommentsRecommender
from implicit_extend.random import RandomRecommender
from implicit_extend.nearest_neighbours_ub import CosineRecommenderUB
from implicit_extend.content_based import ContentBasedRecommender
from implicit_extend.evaluation import ranking_metrics_at_k
from implicit_extend.hybrid import HybridRecommenderUB, HybridRecommenderIB
# Auxiliary functions
from dusa_function_lib import get_rm_train_test_info, build_db_name
from dusa_function_lib import build_directory_city_name
from dusa_function_lib import tunning_and_metrics
from dusa_function_lib import gen_recommendations
from dusa_function_lib import get_n_for_ndcg
from dusa_function_lib import get_item_category_info, get_item_location_info


In [ ]:
# mysql database connection
connection = pymysql.connect(
    host='localhost',
    user='eduardomv',
    password='*****',
    database='participatory_budgeting'
)
cursor = connection.cursor()

cities = ['Cambridge', 'Miami', 'New York']
years = ['2014', '2015', '2016', '2017']
for city in cities:
    city_directory_name = build_directory_city_name(city)
    for year in years:
        database_name = build_db_name(city, year)
        query = f"""
        SELECT userId, itemId, COUNT(*) as num_comments
        FROM 
            ratings r
        WHERE 
            dataset = '{database_name}'
        GROUP BY userId, itemId
        ORDER BY userId ASC, itemId ASC;
        """
        cursor.execute(query)
        
        data = cursor.fetchall()
        
        user_ids = []
        proposal_ids = []
        num_comments = []
        
        for row in data:
            user_ids.append(row[0])
            proposal_ids.append(row[1])
            num_comments.append(row[2])
    
        unique_user_ids = {uid: id_enum for id_enum, uid in enumerate(pd.unique(user_ids))}
        unique_proposal_ids = {pid: idx for idx, pid in enumerate(pd.unique(proposal_ids))}
        
        row_indices = [unique_user_ids[uid] for uid in user_ids] # looks for the new value of the user id
        col_indices = [unique_proposal_ids[pid] for pid in proposal_ids]
        
        # Create the sparse matrix
        matrix = csr_matrix(
            (num_comments, (row_indices, col_indices)),
            shape=(len(unique_user_ids), len(unique_proposal_ids))
        )
        
        city_year_directory_rm_name = f"../../data/rm/{city_directory_name}/{year}"
        save_npz(f"{city_year_directory_rm_name}/num_comm_matrix.npz", matrix)
        print(f"Matrix {database_name} has shape: {matrix.shape}")
        
        user_mapping = pd.DataFrame(list(unique_user_ids.items()), columns=['userId', 'new_userId'])
        proposal_mapping = pd.DataFrame(list(unique_proposal_ids.items()), columns=['itemId', 'new_itemId'])
        
        user_mapping.to_csv(f"{city_year_directory_rm_name}/user_mapping.csv", index=False, sep="|")
        proposal_mapping.to_csv(f"{city_year_directory_rm_name}/item_mapping.csv", index=False, sep="|")

        # Split the matrix data into train and test
        mat_train, mat_test = train_test_split(matrix, train_percentage=0.8, random_state=1)
        mat_train = mat_train.astype(float)
        mat_test = mat_test.astype(float)
        
        save_npz(f"{city_year_directory_rm_name}/num_comm_train.npz", mat_train)
        save_npz(f"{city_year_directory_rm_name}/num_comm_test.npz", mat_test)
        
cursor.close()
connection.close()
print("Done!")

Now we will create some extra files that will help us to work with the data. As before, we will store this data in a sparse matrix format to save memory. The files we will create are:
- `data\category\city\year\item_category_matrix.npz`: sparse matrix with the categories of each proposal. The proposals will be stored in the rows and the categories in the columns. The value of the matrix will be 1 if the proposal belongs to the category and 0 otherwise.
- `data\category\city\year\it_cat_category_mapping.csv`: mapping between each category and its index in the matrix.
- `data\category\city\year\it_cat_item_mapping.csv`: mapping between each itemId and its index in the matrix.

In [ ]:
# mysql database connection
connection = pymysql.connect(
    host='localhost',
    user='eduardomv',
    password='*****',
    database='participatory_budgeting'
)
cursor = connection.cursor()

cities = ['Cambridge', 'Miami', 'New York']
years = ['2014', '2015', '2016', '2017']
for city in cities:
    city_directory_name = build_directory_city_name(city)
    for year in years:
        database_name = build_db_name(city, year)
        query = f"""
        SELECT id, category
        FROM
            items it
        WHERE 
            dataset = '{database_name}'
        """
        cursor.execute(query)
        
        data = cursor.fetchall()
        
        proposal_ids = []
        categories = []
        category_in_proposal = []
        
        for row in data:
            proposal_ids.append(row[0])
            categories.append(row[1])
            category_in_proposal.append(1)
    
        unique_proposal_ids = {pid: idx for idx, pid in enumerate(pd.unique(proposal_ids))}
        unique_categories_ids = {cat: idx for idx, cat in enumerate(pd.unique(categories))}
        
        row_indices = [unique_proposal_ids[pid] for pid in proposal_ids]
        col_indices = [unique_categories_ids[cat] for cat in categories]
        
        # Create the sparse matrix
        matrix = csr_matrix(
            (category_in_proposal, (row_indices, col_indices)),
            shape=(len(unique_proposal_ids), len(unique_categories_ids))
        )
        
        city_year_directory_rm_name = f"../../data/category/{city_directory_name}/{year}"
        save_npz(f"{city_year_directory_rm_name}/item_category_matrix.npz", matrix)
        print(f"Matrix item_category from {database_name} has shape: {matrix.shape}")
        
        proposal_mapping = pd.DataFrame(list(unique_proposal_ids.items()), columns=['itemId', 'new_itemId'])
        category_mapping = pd.DataFrame(list(unique_categories_ids.items()), columns=['category', 'new_categoryId'])
        
        proposal_mapping.to_csv(f"{city_year_directory_rm_name}/it_cat_item_mapping.csv", index=False, sep="|")
        category_mapping.to_csv(f"{city_year_directory_rm_name}/it_cat_category_mapping.csv", index=False, sep="|")
        
        
cursor.close()
connection.close()
print("Done!")

# Clustering to generate "Neighborhoods"
In this section we will create the neighborhoods for each city. These neighborhoods will be used in the location recommendation algorithms. They will be created using the KMeans algorithm. The number of clusters will be defined depending on the silhouette score. We will save the neighborhoods in the following format:
- `../data/location/city/clusters_city.csv`: csv file with the clusters for each city. The file will have the following structure:
    - `dataset`: name of the dataset (city-year)
    - `id`: id of the proposal
    - `latitude`: latitude of the proposal
    - `longitude`: longitude of the proposal
    - `cluster`: id of the cluster
Additionally, we will create a map in html in order to visualize the clusters. It will be saved in the following directory:
- `../../data/location/city/mapa_clusters_city.html`: html file with the map of the clusters.

After generating the `csv` files, we will create some sparse matrices that will be used in the location recommendation algorithms. This sparse matrices will have itemId in the rows and the clusters in the columns. The value of the matrix will be 1 if the proposal belongs to the cluster and 0 otherwise. We will save the matrices in the following files:
- `../../data/location/city/year/item_cluster_matrix.npz`: sparse matrix with the clusters of each proposal of that city and year.
- `../../data/location/city/year/it_loc_item_mapping.csv`: mapping between each itemId and its index in the matrix.
- `../../data/location/city/year/it_loc_cluster_mapping.csv`: mapping between each cluster and its index in the matrix.

In [ ]:
%load_ext autoreload
%autoreload 2
import pymysql
import os
import sys
import matplotlib.cm as cm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import folium
import pandas as pd
from scipy.sparse import csr_matrix, save_npz
from implicit.evaluation import train_test_split

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    python_path = os.path.join(module_path, 'python')
    sys.path.append(python_path)

from dusa_function_lib import build_directory_city_name, build_db_name

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# mysql database connection
connection = pymysql.connect(
    host='localhost',
    user='eduardomv',
    password='*****',
    database='participatory_budgeting'
)
cursor = connection.cursor()

cities = ['Cambridge', 'Miami', 'New York']
years = ['2014', '2015', '2016', '2017']
for city in cities:
    db_city_names = []
    city_directory_name = build_directory_city_name(city)
    for year in years:
        db_city_names.append(build_db_name(city, year))
    
    query = f"""
    SELECT dataset, id, longitude, latitude
    FROM 
        items
    WHERE 
        dataset = '{db_city_names[0]}' OR dataset = '{db_city_names[1]}' OR dataset = '{db_city_names[2]}' OR dataset = '{db_city_names[3]}';
    """
    cursor.execute(query)
    
    data = cursor.fetchall()
    
    databases = []
    user_ids = []
    longitudes = []
    latitudes = []
    
    for row in data:
        databases.append(row[0])
        user_ids.append(row[1])
        longitudes.append(row[2])
        latitudes.append(row[3])
    
    df = pd.DataFrame({'dataset':databases, 'id': user_ids, 'latitude': latitudes, 'longitude': longitudes})

    coords = df[['latitude', 'longitude']].values

    # Determine the optimal number of clusters using silhouette score
    max_k = 10
    best_k = 4
    best_score = -1

    for k in range(4, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42).fit(coords)
        labels_temp = kmeans.labels_
        sil_score = silhouette_score(coords, labels_temp)
        if sil_score > best_score:
            best_score = sil_score
            best_k = k

    print(f"Número óptimo de clústers según silhouette: {best_k} (score={best_score:.2f})")

    kmeans_final = KMeans(n_clusters=best_k, random_state=42)
    df['cluster'] = kmeans_final.fit_predict(coords)

    center_lat = df['latitude'].mean()
    center_lon = df['longitude'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    colormap = cm.get_cmap('tab10', best_k)  # 'tab10', 'Set1', etc. ajustables
    cluster_colors = {}
    for cl in range(best_k):
        rgba = colormap(cl)[:3]
        hexcolor = '#{:02x}{:02x}{:02x}'.format(int(rgba[0]*255),
                                            int(rgba[1]*255),
                                            int(rgba[2]*255))
        cluster_colors[cl] = hexcolor

    for idx, row in df.iterrows():
        cl = row['cluster']
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=cluster_colors[cl],
            fill=True,
            fill_color=cluster_colors[cl],
            fill_opacity=0.8,
        ).add_to(m)

    legend_html = '''
    <div style="position: fixed; 
        bottom: 50px; left: 50px; width: 150px; height: auto; 
        border:2px solid grey; z-index:9999; font-size:14px;">
    &nbsp;<b>Clusters</b><br/>
    '''
    for cl in range(best_k):
        legend_html += f'''&nbsp;<i style="background:{cluster_colors[cl]};width:12px;height:12px;
                        display:inline-block;"></i>
                        &nbsp;Cluster {cl}<br/>'''
    legend_html += '</div>'

    m.get_root().html.add_child(folium.Element(legend_html))

    m.save(f"../../data/location/{city_directory_name}/mapa_clusters_{city_directory_name}.html")
    df.to_csv(f"../../data/location/{city_directory_name}/clusters_{city_directory_name}.csv", index=False)
    
        
cursor.close()
connection.close()
print("Done!")

Número óptimo de clústers según silhouette: 8 (score=0.42)


C:\Users\molin\AppData\Local\Temp\ipykernel_19268\1473451583.py:71: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('tab10', best_k)  # 'tab10', 'Set1', etc. ajustables


Número óptimo de clústers según silhouette: 6 (score=0.50)


C:\Users\molin\AppData\Local\Temp\ipykernel_19268\1473451583.py:71: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('tab10', best_k)  # 'tab10', 'Set1', etc. ajustables


Número óptimo de clústers según silhouette: 8 (score=0.49)


C:\Users\molin\AppData\Local\Temp\ipykernel_19268\1473451583.py:71: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('tab10', best_k)  # 'tab10', 'Set1', etc. ajustables


Done!


In [ ]:
for city in cities:
    city_directory_name = build_directory_city_name(city)
    df = pd.read_csv(f"../../data/location/{city_directory_name}/clusters_{city_directory_name}.csv")
    for year in years:
        database_name = build_db_name(city, year)
        df_year = df[df['dataset'] == database_name]
        proposal_ids = df_year['id'].tolist()
        proposals_clusters = df_year['cluster'].tolist()
        
        unique_proposals_ids = {pid: idx for idx, pid in enumerate(pd.unique(proposal_ids))}
        unique_proposals_clusters = {clus: idx for idx, clus in enumerate(pd.unique(proposals_clusters))}
        
        row_indices = [unique_proposals_ids[pid] for pid in proposal_ids]
        col_indices = [unique_proposals_clusters[clus] for clus in proposals_clusters]
        
        # crear lista de unos de la longitud de la cantidad de propuestas
        cluster_in_proposals = [1] * len(proposal_ids)
        
        matrix = csr_matrix(
            (cluster_in_proposals, (row_indices, col_indices)),
            shape=(len(unique_proposals_ids), len(unique_proposals_clusters))
        )
        
        city_year_location_name = f"../../data/location/{city_directory_name}/{year}"
        save_npz(f"{city_year_location_name}/item_cluster_matrix.npz", matrix) 
        proposal_mapping = pd.DataFrame(list(unique_proposals_ids.items()), columns=['itemId', 'new_itemId'])
        cluster_mapping = pd.DataFrame(list(unique_proposals_clusters.items()), columns=['clusterId', 'new_clusterId'])
        proposal_mapping.to_csv(f"{city_year_location_name}/it_loc_item_mapping.csv", index=False, sep='|')
        cluster_mapping.to_csv(f"{city_year_location_name}/it_loc_cluster_mapping.csv", index=False, sep='|')
        

C:\Users\molin\AppData\Local\Temp\ipykernel_19268\3751535391.py:10: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_proposals_ids = {pid: idx for idx, pid in enumerate(pd.unique(proposal_ids))}
C:\Users\molin\AppData\Local\Temp\ipykernel_19268\3751535391.py:11: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_proposals_clusters = {clus: idx for idx, clus in enumerate(pd.unique(proposals_clusters))}
C:\Users\molin\AppData\Local\Temp\ipykernel_19268\3751535391.py:10: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_proposals_ids = {pid: idx for idx, pid in enumerate(pd.unique(proposal_ids))}
C:\Users\molin\AppData\Local\Temp\ipykernel_19268\3751535391.py:11: FutureW